# TP : Analyse en Composantes Principales (ACP) et choix du clustering

## 1. Introduction

### 🌟 Objectifs du TP
- Comprendre le fonctionnement du PCA (ACP en français)
- Réduire la dimension de données pour les visualiser
- Comparer les résultats de clustering (K-Means et DBSCAN) sur les données projetées

### 🔍 Qu'est-ce que le PCA ?
L'**Analyse en Composantes Principales** (ACP ou PCA pour *Principal Component Analysis*) est une méthode mathématique qui permet de :
- **Réduire le nombre de dimensions** d'un jeu de données,
- En conservant un maximum d'information (**variance**),
- En projetant les points dans un nouvel espace (appelé espace des composantes principales).

Cela permet notamment de **visualiser** des données complexes en 2D, ou de **prétraiter les données** avant un clustering.

### 🧹 Pourquoi prétraiter les données avec le PCA avant un clustering ?
- Lorsque les données ont **beaucoup de dimensions**, certaines peuvent être **corrélées ou peu informatives**.
- Le PCA élimine ces redondances et simplifie la structure des données.
- En réduisant le bruit et la complexité, cela peut :
  - améliorer la qualité des clusters,
  - faciliter le travail de K-Means ou DBSCAN,
  - réduire les temps de calcul.

En résumé, le PCA sert souvent de **filtre intelligent** avant de lancer un algorithme de regroupement.

## 2. Test avec un jeu de données multidimensionnel

Jusqu'ici, on n'a travaillé qu'avec des données ne comportant que deux dimensions pour une meilleure visibilité. Qu'en est-il des données mutidimensionnelles ?

In [ ]:
# Installation des bibliothèques
!pip install pandas numpy scikit-learn matplotlib seaborn

### 📁 2.1 Génération de données (5 dimensions)

In [ ]:
from sklearn.datasets import make_classification
import pandas as pd
import matplotlib.pyplot as plt

# Création d'un jeu avec 5 caractéristiques
X, y = make_classification(n_samples=300, n_features=5, n_redundant=0, 
                           n_clusters_per_class=1, random_state=42)

# Conversion en DataFrame
df = pd.DataFrame(X, columns=[f"Feature_{i+1}" for i in range(X.shape[1])])
df.head()  # Affichage des attributs et des 5 premiers enregistrements

### 🔌2.2  Détermination des composantes et variance par l'algorithme PCA 

L'algorithme PCA va permettre de diminuer le nombre de dimensions, de 5 à 2 ici.

In [ ]:
from sklearn.decomposition import PCA

# Réduction à 2 composantes principales
pca = PCA(n_components=2)
X_reduit = pca.fit_transform(X)

Chaque **composante** est une combinaison linéaire de plusieurs critères, et c’est cette combinaison entière qui explique une part de la variance.

**La variance** :

- ❌ La variance expliquée par une composante ne reflète pas directement le poids d’un critère (ou variable).
- ✅ Le pourcentage de variation totale des données capté par chaque **composante principale**.



🎓 **<u>Exemple</u>** :
Si `Composante 1` explique 60% de la variance, cela signifie qu’en projetant toutes les données sur cet axe, on conserve 60 % de l'information globale. Mais cela ne dit pas quel critère (ex : revenu, âge...) est responsable de ce 60 %.




In [ ]:
# Détermination de la variance de chaque composante 
pca = PCA(n_components=2)  # 2 composantes
X_reduit = pca.fit_transform(X)
print(pca.explained_variance_ratio_)


On voit que seulement **55% (32% + 23%) de l'information** est conservée : il va falloir utiliser plus de composants.

In [ ]:
# Détermination de la variance de chaque composante 
pca = PCA(n_components=3) # 3 composantes
X_pca3 = pca.fit_transform(X)
print(pca.explained_variance_ratio_)

On passe maintenant à **75% de l'information** de conservée : c'est bien mieux :).
Pour visualiser tout cela, on va afficher un **graphique par paire de composants**, ce sera plus lisible.

In [ ]:
from sklearn.decomposition import PCA

# Réduction à 3 composantes principales
pca = PCA(n_components=3)
X_pca3 = pca.fit_transform(X)


# Affichage des 3 combinaisons de composantes principales
pairs = [(0, 1), (0, 2), (1, 2)]
for i, j in pairs:
    plt.figure()
    plt.scatter(X_pca3[:, i], X_pca3[:, j], s=50)
    plt.xlabel(f"Composante {i+1}")
    plt.ylabel(f"Composante {j+1}")
    plt.title(f"Projection : Composantes {i+1} et {j+1}")
    plt.grid(True)
    plt.show()

### 📊 2.3 Comparaison avec K-Means / DBSCAN

**<u>Remarque</u>** : on pourra toujours appliquer la `méthode du coude (K-Means)` et le `k-distance-plot (DBSCAN)` pour déterminer les **meilleurs paramètres** (n_cluster et eps / min_samples).

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import numpy as np


### 🔎 Application de K-Means 
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)  # Meilleur k observé
k_labels = kmeans.fit_predict(X_pca3)

# Affichage 2D selon les 2 premières composantes
plt.scatter(X_pca3[:, 0], X_pca3[:, 1], c=k_labels, cmap="viridis", s=50)
plt.title("K-Means sur données PCA (3 composantes)")
plt.show()



### 🔎 DBSCAN
dbscan = DBSCAN(eps=0.813, min_samples=3)  # Meilleure estimation eps/min_samples
d_labels = dbscan.fit_predict(X_pca3)

# Affichage 2D selon les 2 premières composantes
plt.scatter(X_pca3[:, 0], X_pca3[:, 1], c=d_labels, cmap="plasma", s=50)
plt.title("DBSCAN sur données PCA (3 composantes)")
plt.show()

**<u>Remarque</u>** : on peut aussi proposer un **affichage en 3D** à titre informatif.

In [ ]:
### 🧊 Affichage 3D avec matplotlib
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_pca3[:, 0], X_pca3[:, 1], X_pca3[:, 2], c=k_labels, cmap='viridis', s=50)
ax.set_xlabel("Composante 1")
ax.set_ylabel("Composante 2")
ax.set_zlabel("Composante 3")
ax.set_title("Visualisation 3D des données PCA + K-Means")
plt.show()

### 🎯2.4. Comparaison / Questions

- Qu'observe-t-on ? *On observe que les données projetées en 2D conservent une structure qui permet de distinguer plusieurs groupes.
  Les points forment des regroupements visibles, bien que la séparation ne soit pas toujours nette.*
  
  
- Le **clustering** change-t-il entre K-Means et DBSCAN ? *Oui. K-Means impose un nombre fixe de clusters et découpe l'espace de façon circulaire. DBSCAN, lui, détecte automatiquement les groupes selon la densité des points et peut identifier du bruit.*


- Quels sont les **avantages / inconvénients** visibles ? *K-Means détermine efficacement 3 clusters (on pourrait essayer avec d'autres valeurs). DBSCAN n'est pas fiable ici, les données ne sont pas assez denses et/ou trop espacées.*


- Que nous apprend la projection PCA sur la forme des données ? *Elle montre que les données initiales à 5 dimensions peuvent être bien représentées dans un espace ici à 3 dimensions tout en conservant une bonne partie de la structure. Cela facilite la visualisation et le clustering.*


## 3. TP : Catégoriser des patients selon le risque de cancer.


## 🎯 Objectifs du TP
- Analyser un jeu de données contenant des informations sur des **facteurs de risque de cancer**.
- Réduire la dimension des données à 2 ou 3 composantes avec PCA.
- Visualiser les individus dans l'espace réduit.
- Explorer les regroupements potentiels par clustering (`K-Means` ou `DBSPAN`).

Un fichier de données contient 2000 clients fictifs avec les informations suivantes :
- `âge` (en années)
- `poids` (en kg)
- `antécédents familiaux` (oui / non)
- `tabac` (oui / non)
- `alcool` (oui / non)
- `activité sportive`(faible, moyenne, intense)

### 📥 3.1 – Chargement des données avec pandas

In [ ]:
# Chargement depuis un fichier CSV (à placer dans le même dossier que ce notebook)
import pandas as pd

df = pd.read_csv("donnees_cancer_pca.csv", encoding="utf-8", encoding_errors="ignore")  # Noter le UTF-8 :)
print(df.head())

### 🔄 3.2 – Encodage des données

Avant d'appliquer le PCA, il faut convertir les variables non numériques en format numérique. On utilise :

- `LabelEncoder` pour les **variables ordinales**, c'est-à-dire celles qui ont un ordre logique entre les modalités (ex : "Faible" < "Modérée" < "Intense").
- `get_dummies` pour les **variables nominales**, qui représentent des catégories sans ordre (ex : "Oui" ou "Non").


**<u>Remarques</u>** :
- On pourrait utiliser tout le temps `LabelEncoder` même à la place de `get_dummies`, attention à ne pas introduire **d'ordre artificiel**.
- `get_dummies` permet d'encoder plusieurs attributs en même temps contrairement à `LabelEncoder`.


#### 🧠 **Fonctionnement** de `get_dummies` 

Quand `get_dummies` encode une variable binaire comme "*Fumeur*" avec les modalités "*Oui*" et "*Non*", il crée deux colonnes :

| Fumeur_Oui | Fumeur_Non |
|------------|------------|
| 1          | 0          |
| 0          | 1          |

Avec `drop_first=True`, seule la colonne `Fumeur_Oui` est conservée. Cela évite les **redondances** et les **liens entre attributs** qui pourraient perturber des algorithmes contre les régressions et le PPCA

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt


# On crée une copie de df pour travailler dessus sans modifier l'original
# Cela évite d'altérer le DataFrame de base si on veut le réutiliser ensuite 
# (l'encodage rajoute des colonnes et modifie les valeurs).
X = df.copy()


# Encodage de l'activité physique (ordinale)
le = LabelEncoder()
X["Activité_Physique"] = le.fit_transform(X["Activité_Physique"])

# Encodage one-hot des autres variables catégorielles (non ordonnées)
# Le drop_first = True permet d'éviter les redondances 
X_encoded = pd.get_dummies(X, columns=["Fumeur", "Consommation_Alcool", "Antécédents_Familiaux"], drop_first=True)

### 🧠 3.4  Analyse en composantes principales (PCA)

Il faut déterminer le nombre de composantes optimal pour le PCA. Mais avant cela, il faut **standardiser** les critères, c'est à dire leur donner un **poids égal** (les composantes sont des combinaisons linéaires des critères fournis, ainsi, un âge entre 40 et 70 ans pèserait bien plus qu'un 0 / 1 caractérisant un fumeur ou non).

✅ Ce que fait `StandardScaler` :
Il transforme chaque colonne pour qu’elle ait :
- une moyenne = 0
- un écart type = 1

Autrement dit, il met toutes **les variables sur la même échelle**, ce qui rend leur importance équivalente dans le PCA.

✅ Nombre de **composantes nécessaires** au PCA : un minimum de **75% d'informations retenues** est attendu ici.

In [ ]:
# Mise des variables à la même échelle
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded) # Entraînement du scaler


# TEST : PCA avec 2 composantes (avec résultats)
################### A COMPLETER ########################



# TEST : PCA avec 3 composantes (avec résultats)
################### A COMPLETER ########################



### 📌Questions

- 1. **Compléter** le code manquant permettant de tester le PCA pour 2 et 3 composantes. On notera **<u>obligatoirement</u>** `X_pca` les données travaillées après l'entraînement du modèle PCA.
- 2. **Conclure** sur le nombre de composantes.

**<u>Aide</u>** : on verra ici que *2 composantes* sont largement suffisantes (95% d'informations retenues).

### 🧠 3.5  Application du PCA et détermination du meilleur modèle K-Means / DBSCAN


In [ ]:
# Application de la PCA avec le bon nombre de composantes
################### A COMPLETER ########################


# Affichage 2D 
plt.scatter(X_pca[:, 0], X_pca[:, 1], s=50, alpha=0.7)
plt.xlabel("Composante 1")
plt.ylabel("Composante 2")
plt.title("Projection PCA (2 composantes)")
plt.grid(True)
plt.show()

### 📌Questions

- 1. **Compléter** le code manquant.
- 2. Quel modèle entre le **K-Means** et le **DBSCAN** pourrait-on appliquer ici ? Justifier. <u>ON FERA UNE ANALYSE SEULEMENT QUALITATIVE</u> (pas de tests). 
- 3. En déduire <u>qualitativement</u> le (ou les) paramètres `n_clusters` (K_Means) ou `min_samples` / `eps` (DBSCAN)

### 🧠 3.6  Application du PCA et détermination du meilleur modèle entre K-Means et DBSCAN

In [ ]:
# Application du meilleur modèle avec les bons critères
################ A COMPLETER ###################


# Affichage des résultat
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=k_labels, cmap='Set2', s=50)

# Ajouter les numéros de clusters en annotation au centre de chaque groupe
for cluster_id in np.unique(k_labels):
    # coordonnées du centre de gravité du cluster
    x_mean = X_pca[k_labels == cluster_id, 0].mean()
    y_mean = X_pca[k_labels == cluster_id, 1].mean()
    plt.text(x_mean, y_mean, str(cluster_id), fontsize=14, weight='bold',
             ha='center', va='center', color='black', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'))

plt.title("Algorithme sur données PCA avec numéros de clusters")
plt.xlabel("Composante 1")
plt.ylabel("Composante 2")
plt.grid(True)
plt.show()

### 📌Question

A priori, combien de **groupes de patients** peut-on proposer à partir des résultats ?  

### 🧠 3.7  Prédiction de l'appartenance d'un patient à une catégorie 

On a vu qu'avec l'algorithme **DBSCAN**, on devait utiliser **KNN** pour **estimer la catégorie d'appartenance** d'une nouvelle donnée. 
Voici la démarche à suivre avec l'algorithme **K_Means**.

#### 📝 Résumé des étapes de prédiction pour un nouveau patient

| Étape | Description |
|-------|-------------|
| 1️⃣ Encodage de l'activité physique | Utiliser le `LabelEncoder` déjà entraîné (`le`) sur la colonne `"Activité_Physique"` |
| 2️⃣ Encodage des variables nominales | Utiliser `pd.get_dummies` avec `drop_first=True` pour les variables comme `"Fumeur"`, `"Alcool"`... |
| 3️⃣ Ajustement des colonnes | Ajouter les colonnes manquantes pour correspondre à `X_encoded`, puis réordonner les colonnes |
| 4️⃣ Standardisation | Appliquer le `scaler.transform(...)` entraîné précédemment sur `X_encoded` |
| 5️⃣ Réduction de dimension | Appliquer `pca.transform(...)` sur les données standardisées du patient |
| 6️⃣ Prédiction du cluster | Utiliser `kmeans.predict(...)` pour obtenir le numéro du cluster associé |


**<u>Exemple</u>** : On teste ici pour un patient avec ces caractéristiques : *Âge = 65 ans, Poids = 80 kg, Antécédents = Oui, Tabac = Non, Alcool = Oui, Activité = Faible*

In [ ]:
# Caractéristiques du patient :
# Âge = 65 ans, Poids = 80 kg, Antécédents = Oui, Tabac = Non, Alcool = Oui, Activité = Faible

patient = pd.DataFrame([{
    "Âge": 65,
    "Poids": 80,
    "Fumeur": "Non",
    "Consommation_Alcool": "Oui",
    "Antécédents_Familiaux": "Oui",
    "Activité_Physique": "Faible"
}])

# Étape 1 : Encodage de l'activité physique avec le même LabelEncoder que précédemment
patient["Activité_Physique"] = le.transform(patient["Activité_Physique"])

# Étape 2 : Encodage one-hot des autres variables catégorielles (comme dans le TP)
patient_encoded = pd.get_dummies(patient, drop_first=True)

# Étape 3 : Ajouter les colonnes manquantes pour correspondre à X_encoded
for col in X_encoded.columns:
    if col not in patient_encoded.columns:
        patient_encoded[col] = 0
patient_encoded = patient_encoded[X_encoded.columns]  # Reordonner les colonnes

# Étape 4 : Standardiser avec le scaler déjà entraîné
patient_scaled = scaler.transform(patient_encoded)

# Étape 5 : Réduction avec le PCA déjà entraîné
patient_pca = pca.transform(patient_scaled)

# Étape 6 : Prédiction avec le modèle KMeans
cluster_pred = kmeans.predict(patient_pca)[0]
print(f"✅ Le patient est classé dans le cluster {cluster_pred}")


### 🧠 3.7  Analyse des résultats

Il est important de comprendre qu'avec le PCA, les **composantes** sont une **combinaison linéaires des critères** relevés : on ne peut donc pas analyser directement les groupes obtenus.

En revanche, on peut cibler les types de personnes composants les clusters : dans l'exemple ci-dessus, 
chaque cluster correspond probablement à un **profil de risque** :
  - 🟩 Un cluster peut regrouper des patients **jeunes, actifs, non-fumeurs, sans antécédents** → risque faible
  - 🟥 Un autre cluster peut représenter des patients **plus âgés, fumeurs, avec antécédents** → risque élevé
  - 🟦 Le troisième est un **profil intermédiaire** entre les deux.

In [ ]:
import seaborn as sns


# Ajout des labels K-Means aux données d'origine
X_labeled = df.copy()
X_labeled["Cluster"] = k_labels


# Projection 2D (composantes 1 et 2)
plt.scatter(X_pca[:, 0], X_pca[:, 1], s=50, alpha=0.7)
plt.xlabel("Composante 1")
plt.ylabel("Composante 2")
plt.title("Projection PCA (2 composantes)")
plt.grid(True)
plt.show()


# Barplot de la consommation de tabac par cluster
plt.figure(figsize=(6, 4))
sns.barplot(
    data=X_labeled.groupby("Cluster")["Fumeur"].apply(lambda x: (x == "Oui").mean()).reset_index(),
    x="Cluster", y="Fumeur"
)
plt.title("Proportion de fumeurs par cluster")
plt.ylabel("Proportion de fumeurs")
plt.grid(True)
plt.show()


# Barplot de la consommation d'alcool par cluster
plt.figure(figsize=(6, 4))
sns.barplot(
    data=X_labeled.groupby("Cluster")["Consommation_Alcool"].apply(lambda x: (x == "Oui").mean()).reset_index(),
    x="Cluster", y="Consommation_Alcool"
)
plt.title("Proportion de consommateurs d'alcool par cluster")
plt.ylabel("Proportion d'alcool")
plt.grid(True)
plt.show()


# Barplot des antécédents familiaux par cluster
plt.figure(figsize=(6, 4))
sns.barplot(
    data=X_labeled.groupby("Cluster")["Antécédents_Familiaux"].apply(lambda x: (x == "Oui").mean()).reset_index(),
    x="Cluster", y="Antécédents_Familiaux"
)
plt.title("Proportion d'antécédents familiaux par cluster")
plt.ylabel("Proportion avec antécédents")
plt.grid(True)
plt.show()


# Barplot de l'activité physique par cluster
plt.figure(figsize=(6, 4))
sns.barplot(
    data=X_labeled.groupby("Cluster")["Activité_Physique"].value_counts(normalize=True).rename("Proportion").reset_index(),
    x="Cluster", y="Proportion", hue="Activité_Physique"
)
plt.title("Répartition de l'activité physique par cluster")
plt.grid(True)
plt.show()



# Boxplot de l'âge par cluster
plt.figure(figsize=(6, 4))
sns.boxplot(data=X_labeled, x="Cluster", y="Âge")
plt.title("Distribution de l'âge par cluster")
plt.grid(True)
plt.show()

# Boxplot du poids par cluster
plt.figure(figsize=(6, 4))
sns.boxplot(data=X_labeled, x="Cluster", y="Poids")
plt.title("Distribution du poids par cluster")
plt.ylabel("Poids (kg)")
plt.grid(True)
plt.show()

### 🧠 Résumé : interprétation des clusters avec les barplots et boxplots

Pour comprendre ce que représente chaque cluster (faible, moyen ou fort risque de cancer), on utilise :

| Type de graphique        | Variables concernées                            | Ce qu'on observe                                                         |
|--------------------------|--------------------------------------------------|--------------------------------------------------------------------------|
| 📦 Boxplot               | `Âge`, `Poids`                                   | Repère les groupes plus âgés ou corpulents                              |
| 📊 Barplot (proportions) | `Fumeur`, `Consommation_Alcool`, `Antécédents_Familiaux` | Observe la part de patients exposés à des facteurs de risque           |
| 📊 Barplot (répartition) | `Activité_Physique`                   | Compare les modalités (ex : actif / non) dans chaque cluster             |


### 📌Question

Classer les clusters en fonction des **risques de cancers**.

### 🌲 3.8 Influence des différents critères

Une fois les clusters identifiés et associés à des niveaux de risque, on peut entraîner un **modèle supervisé** (Random Forest ici) pour déterminer quels critères influencent le plus le classement des patients.


#### 📘 Résumé – Déterminer l’importance des critères dans le risque de cancer (Random Forest)

Pour savoir **quels critères influencent le plus le classement des patients** par niveau de risque, on suit ces étapes :

| Étape | Description                                                                 |
|-------|-----------------------------------------------------------------------------|
| 1️⃣    | Associer chaque **cluster** (0, 1, 2) à un **niveau de risque** (`faible`, `modéré`, `élevé`) |
| 2️⃣    | Ajouter cette information dans le tableau principal (`df`)                |
| 3️⃣    | Utiliser le DataFrame encodé (`X_encoded`) comme **base d’apprentissage** |
| 4️⃣    | Entraîner un modèle **RandomForestClassifier** avec `Cluster` comme cible  |
| 5️⃣    | Extraire l’**importance des variables** via `.feature_importances_`        |
| 6️⃣    | Afficher un **histogramme horizontal** pour visualiser les critères les plus déterminants |

✅ Ce processus permet d’**interpréter les décisions du modèle** et de **hiérarchiser les facteurs de risque** en fonction de leur contribution.



In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import matplotlib.pyplot as plt

# Étape 1 : Associer les clusters à des niveaux de risque
# (adapte si nécessaire selon ton analyse visuelle)
cluster_risque = {
    0: "Modéré",
    1: "Faible",
    2: "Élevé"
}

# Étape 2 : Créer les colonnes "Cluster" et "Niveau_Risque"
df["Cluster"] = k_labels
df["Niveau_Risque"] = df["Cluster"].map(cluster_risque)

# On vérifie que l'ajout des colonnes est effectif
print(df.head())


# Étape 3 : Utiliser les données entraînées précédemment 
X_features = X_encoded.copy()
y_risque = df["Cluster"]  

# Étape 4 : Entraîner une Random Forest pour analyser l'importance des critères
rf = RandomForestClassifier(random_state=42)
rf.fit(X_features, y_risque)

# Étape 5 : Extraire l’importance des variables 
importances = pd.Series(rf.feature_importances_, index=X_features.columns)
importances.sort_values(ascending=True).plot(kind="barh", figsize=(8, 6))

# Étape 6 : Les afficher sous forme d’histogramme
plt.title("Importance des critères pour prédire le niveau de risque (Random Forest)")
plt.xlabel("Importance")
plt.grid(True)
plt.tight_layout()
plt.show()
